<a href="https://colab.research.google.com/github/cbeckler/eco_cancer_incidence_rates/blob/main/db_tables_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://ppa.launchpad.net/c2d4u.team/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-20 20:42:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2023-01-20 20:42:08 (1.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# start spark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("save_tables").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#connect to database

import pandas as pd

conn_path = "/content/drive/MyDrive/Data/db_conn_info.csv"

conn_info = pd.read_csv(conn_path)

db_name = conn_info['db_name'][0]

db_pw = conn_info['password'][0]

db_host = conn_info['host'][0]

db_port = conn_info['port'][0]

db_user = conn_info['user'][0]

In [6]:
from sqlalchemy import create_engine

engine = create_engine(f'postgresql://{db_name}:{db_pw}@{db_host}:{db_port}/{db_user}')

In [7]:
viz_df = pd.read_sql('select * from visualization_data', engine)

viz_df.head()

,FIPS,recent_trend,prediction_trend,O3_max_pred,PM25_max_pred
0,01001,stable,stable,41.001571,12.239594
1,01003,stable,stable,45.070022,11.024418
2,01005,stable,stable,41.139416,11.688170
3,01007,stable,stable,40.505549,12.280141
4,01009,stable,stable,42.714107,13.498799


In [19]:
#viz_df.to_csv("/content/drive/MyDrive/Data/visualization_data.csv", index=False)

In [8]:
pred_df = pd.read_sql('select * from prediction_results', engine)

pred_df.columns = ['FIPS', 'future_prediction_trend']

pred_df.head()

,FIPS,future_prediction_trend
0,01001,stable
1,01003,stable
2,01005,stable
3,01007,stable
4,01009,stable


In [9]:
#pred_df.to_csv("/content/drive/MyDrive/Data/prediction_results.csv", index=False)

In [10]:
import json
# load data using Python JSON module
with open("/content/drive/MyDrive/Data/county_geojson.json", encoding='latin-1') as data_file:                           
    county_data = json.load(data_file)

# Flatten data
county_df = pd.json_normalize(county_data, record_path =['features'])

In [11]:
county_df['FIPS'] = county_df['properties.STATE'].astype(str) + county_df['properties.COUNTY'].astype(str)

county_df.head()

,type,properties.GEO_ID,properties.STATE,properties.COUNTY,properties.NAME,properties.LSAD,properties.CENSUSAREA,geometry.type,geometry.coordinates,FIPS
0,Feature,0500000US01029,01,029,Cleburne,County,560.100,Polygon,"[[[-85.388717, 33.913044], [-85.380885, 33.873...",01029
1,Feature,0500000US01031,01,031,Coffee,County,678.972,Polygon,"[[[-86.030441, 31.618939], [-86.004085, 31.619...",01031
2,Feature,0500000US01037,01,037,Coosa,County,650.926,Polygon,"[[[-86.00928, 33.101638], [-86.00917, 33.09026...",01037
3,Feature,0500000US01039,01,039,Covington,County,1030.456,Polygon,"[[[-86.348506, 30.994339], [-86.35023, 30.9943...",01039
4,Feature,0500000US01041,01,041,Crenshaw,County,608.840,Polygon,"[[[-86.146992, 31.680455], [-86.147114, 31.663...",01041


In [12]:
data_df = viz_df.merge(pred_df, on='FIPS', how='right')

data_df['FIPS'] = data_df['FIPS'].astype(str)

data_df.head()

,FIPS,recent_trend,prediction_trend,O3_max_pred,PM25_max_pred,future_prediction_trend
0,01001,stable,stable,41.001571,12.239594,stable
1,01003,stable,stable,45.070022,11.024418,stable
2,01005,stable,stable,41.139416,11.688170,stable
3,01007,stable,stable,40.505549,12.280141,stable
4,01009,stable,stable,42.714107,13.498799,stable


In [17]:
final_df = county_df.merge(data_df, left_on='FIPS', right_on='FIPS', how='left')

final_df.head()

,type,properties.GEO_ID,properties.STATE,properties.COUNTY,properties.NAME,properties.LSAD,properties.CENSUSAREA,geometry.type,geometry.coordinates,FIPS,recent_trend,prediction_trend,O3_max_pred,PM25_max_pred,future_prediction_trend
0,Feature,0500000US01029,01,029,Cleburne,County,560.100,Polygon,"[[[-85.388717, 33.913044], [-85.380885, 33.873...",01029,stable,stable,42.632166,12.584983,stable
1,Feature,0500000US01031,01,031,Coffee,County,678.972,Polygon,"[[[-86.030441, 31.618939], [-86.004085, 31.619...",01031,stable,stable,40.746204,11.234159,stable
2,Feature,0500000US01037,01,037,Coosa,County,650.926,Polygon,"[[[-86.00928, 33.101638], [-86.00917, 33.09026...",01037,stable,stable,41.526183,11.995925,stable
3,Feature,0500000US01039,01,039,Covington,County,1030.456,Polygon,"[[[-86.348506, 30.994339], [-86.35023, 30.9943...",01039,stable,stable,40.768789,10.987087,stable
4,Feature,0500000US01041,01,041,Crenshaw,County,608.840,Polygon,"[[[-86.146992, 31.680455], [-86.147114, 31.663...",01041,stable,stable,40.666815,11.129129,stable


In [18]:
len(final_df)

3221

In [20]:
final_df.FIPS.nunique()

3221

In [21]:
final_df.to_csv('/content/drive/MyDrive/Data/map_data.csv', index=False)